# Exemplo 1: Using Pydantic for Data Validation

Instalando biblioteca pydantic[email], pois estava dando um erro ao executar
a class Role, devido ao uso do EmailStr

In [1]:
!pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 7.0 MB/s eta 0:00:00


Abaixo temos a importação de módulos e classes que ajudam na criação de modelos de dados validados (pydantic) (typing, any aceitando qualquer tipo de dado) e na definição de enumerações (enum, sendo auto e IntFlag).

In [2]:
from enum import auto, IntFlag
from typing import Any
from pydantic import (BaseModel,
                      EmailStr, # tipos já válidos da biblioteca pydantic
                      Field,
                      SecretStr,
                      ValidationError,
                      )

Abaixo temos duas classes um Role enum, a qual é um int e pode ser um author, editor, developer ou admin que pode ser qualquer um dos três. A user class que herda BaseModel, que foi importada do pydantic e é usada para realizar o básico da validação do pydantic, os campos que fazem parte da classe user, ou seja, o que o usuário deve ter são name ( o qual têm tipo de anotação str e um Field com o exemplos de nomes válidos), email ( usa o tipo EmailStr que veio da biblioteca pydantic e também possuí um Field que contêm alguns parâmetros como exemplos válidos, uma descrição sobre o que é e se deve ficar congelado sem poder ser alterado depois ou não, o "frozen"), password (O mesmo vale para a senha, a qual têm o tipo SecretStr que veio da biblioteca pydantic, o qual quando é imprimido exibe ****** e além disso password possui exemplo e descrição) e role ( que recebe os valores enumerados provenientes da classe Role).

In [3]:
class Role(IntFlag):
  Author = auto()
  Editor = auto()
  Developer = auto()
  Admin = Author | Editor | Developer

class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["example@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(default=None, description="The role of the user")

Abaixo temos definida uma função validação (validate), a qual chama o modelo de validação e nos permite verificar se os dados estão de acordo com a estrutura de usuário definida anteriormente. Logo se estiverem o usuário é imprimido se não imprime que o usuário é inválido e imprime os erros.

In [4]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid")
        for error in e.errors():
            print(error)

Abaixo temos um exemplo de dados corretos (good_data) e incorretos (bad_data) para testar a função validate e verificar se os dados estão de acordo com a estrutura de usuário definida em User.

In [5]:
def main() -> None:
    good_data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }
    bad_data = {"email": "<bad data>", "password": "<bad data>"}

    validate(good_data)
    validate(bad_data)

Quando executado obtemos para o good_data tudo certo e para o bad_data recebemos um erro de que não há um name e de que ambos os valores de email e de password não são válidos. Logo abaixo temos referências da documentação para entender o erro.

In [6]:
if __name__ == "__main__":
  main()

name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '<bad data>', 'password': '<bad data>'}, 'url': 'https://errors.pydantic.dev/2.10/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '<bad data>', 'ctx': {'reason': 'An email address must have an @-sign.'}}


# Exemplo 2: Custom Validators

A diferença em relação ao exemplo anterior é o módulo hashlib para a implementação de criptografia e o re para usar regex.

In [7]:
import enum
import hashlib
import re
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

Expressões regulares para validar nomes e senhas

In [8]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

Abaixo temos o class Role e o Class user modificados, primeiramente sobre o class Role agora ele está recebendo diretamente valores int para cada Role, em vez de auto como no código anterior. Já a class User teve modificações quanto aos objetos pydantic adicionados, os quais são validações personalizadas, as quais não estão presentes na biblioteca pydantic, usando fields validators decorators, o primero é de nome que é definido um valor de classe, onde está sendo verificado pelo regex se o valor corresponde ao obtido e se não é gerado um valor de erro de que o nome deve conter letras e pelo menos 2 caracteres, além disso também não temos um mode nele. Já no validator de field temos um "mode=before", onde pode-se especificar o tipo de entrada que se deseja trabalhar e no caso da função validate_role pode ser um valor inteiro, string ou enum e então tenta-se converter o valor pra um tipo Role e se não funcionar é emitido um erro. Além disso vale citar que ao usar o modo before essas funções de validação deverão ser metódos de classe (No caso do "mode=after", tratam-se de metódos de instância regulares, podendo só passar o self e trabalhar com ele). Logo em seguida temos objeto model_validator, que permite validar todos os ddadfos de uam vez, o qual é útil para verificar se o usuário não está usando o seu name contido no password, onde é usado um dicionário, além disso vale citar que também há o uso de regex para verificar a senha, a qual vai ser inválida se não ter 8 caracteres, 1 maiúscula, 1 minúscula e 1 número, e antes de finalizar temos um hash da senha antes de armzená-la na instância.

In [9]:
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8


class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(
        default=None, description="The role of the user", examples=[1, 2, 4, 8]
    )

    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

Abaixo temos a função validate que vai receber um dicionário com dados para testar se esses são válidos, sendo good_data, bad_data, bad_role, bad_name, duplicados e dados faltantes.

In [10]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)


def main() -> None:
    test_data = dict(
        good_data={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )

    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()

Abaixo podemos observar que o good_data é válido, que o bad_role possui um erro na validação do usuário, que o bad_data possui uma senha inválida que não segue as recomendações, o mesmo pode-se dizer do bad_name, o duplicate usa seu nome de usuário na senha e o missing não t6em dado algum.

In [11]:
if __name__ == "__main__":
    main()

good_data
name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid:
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_data
User is invalid:
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_name
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://

# Exemplo 3: Custom Serilization

In [12]:
import enum
import hashlib
import re
from typing import Any, Self

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
)

Expressões regulares para validar nomes e senhas

In [13]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

A diferença entre o trecho abaixo e os anteriores é o de que Role agora possui um User que recebe zero e que a class User agora conta com um parâmetro novo em field em password, o parâmetro exclude, ou seja, quando o objeto é serializado a senha não é incluída. Incluise agora temos serializers além dos validators, oferecendo mais controle, como em field_serializer de que quando usado um JSON em role, deve-se retornar o nome para poder armazená-lo em formato JSON, o mesmo vale para o model_serializer que agora conta com o "mode=wrap".

In [14]:
class Role(enum.IntFlag):
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8


class User(BaseModel):
    name: str = Field(examples=["Example"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user", exclude=True
    )
    role: Role = Field(
        description="The role of the user",
        examples=[1, 2, 4, 8],
        default=0,
        validate_default=True,
    )

    @field_validator("name")
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user_pre(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

    @model_validator(mode="after")
    def validate_user_post(self, v: Any) -> Self:
        if self.role == Role.Admin and self.name != "Arjan":
            raise ValueError("Only Arjan can be an admin")
        return self

    @field_serializer("role", when_used="json")
    @classmethod
    def serialize_role(cls, v) -> str:
        return v.name

    @model_serializer(mode="wrap", when_used="json")
    def serialize_user(self, serializer, info) -> dict[str, Any]:
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)

Novamente temos a função main com um dicionário para realizar os testes de validação. Onde há a serialização de um dicionário usando model_dump e de uma string JSON. Além disso já que estamos usando o exlude pode-se exluir role como no caso de "user.model_dump(exclude=["role"...."

In [15]:
def main() -> None:
    data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
        "role": "Admin",
    }
    user = User.model_validate(data)
    if user:
        print(
            "The serializer that returns a dict:",
            user.model_dump(),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a JSON string:",
            user.model_dump(mode="json"),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a json string, excluding the role:",
            user.model_dump(exclude=["role"], mode="json"),
            sep="\n",
            end="\n\n",
        )
        print("The serializer that encodes all values to a dict:", dict(user), sep="\n")

Executando o exemplo temos: o serilizador do dicionário, retornando o role como um valor enum (4), a string json tranformada em string, a string json sem função removendo role e o exemplo que codifica todos os valores do dicionário.

In [16]:
if __name__ == "__main__":
    main()

The serializer that returns a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'role': <Role.Admin: 4>}

The serializer that returns a JSON string:
{'name': 'Arjan', 'role': 'Admin'}

The serializer that returns a json string, excluding the role:
{'name': 'Arjan', 'email': 'example@arjancodes.com'}

The serializer that encodes all values to a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>}


# Exemplo 4: Fast API

Instalando o FastAPI

In [17]:
%pip install FastAPI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00


Abaixo temos a importação de datetime para trablhar com datas e horários, Optional (ou seja o campo pode ser None), uuid4 que gera um identificador único aleatório e a própia FastApi. Ao final temos a instância app = FASTAPI para poder adicionar rotas.

In [18]:
from datetime import datetime
from typing import Optional
from uuid import uuid4

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

app = FastAPI()

A diferença desse trecho abaixo em relação aos anteiores da classe User, reside no fato de que este conta conta com opções extras, como por exemplo a configuração que proíbe a adição de atributos extras, ou seja, não é desejável que a API seja chamada, apenas que sejam adicionados vários atributos ao body. Como anteriormente temos name, email, mas agora esses fazem parte de uma API, além desses há atributos adicionais como uma lista de amigos, um ID, uma data e hora opcional e assim como no exemplo anterior podem ser usados serializadores para especificar como retornar esse objeto na API. Abaixo temos o uso da FastAPI onde é declarado um endpoint, para a obtenção de usuários e outro para a criação de usuários, respectivamente "@app.get" e "@app.post", onde ao receber como argumento User a FastAPI vai garantir que seja feita a validação.

In [19]:
class User(BaseModel):
    model_config = {
        "extra": "forbid",
    }
    __users__ = []
    name: str = Field(..., description="Name of the user")
    email: EmailStr = Field(..., description="Email address of the user")
    friends: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of friends"
    )
    blocked: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of blocked users"
    )
    signup_ts: Optional[datetime] = Field(
        default_factory=datetime.now, description="Signup timestamp", kw_only=True
    )
    id: UUID4 = Field(
        default_factory=uuid4, description="Unique identifier", kw_only=True
    )

    @field_serializer("id", when_used="json")
    def serialize_id(self, id: UUID4) -> str:
        return str(id)


@app.get("/users", response_model=list[User])
async def get_users() -> list[User]:
    return list(User.__users__)


@app.post("/users", response_model=User)
async def create_user(user: User):
    User.__users__.append(user)
    return user


@app.get("/users/{user_id}", response_model=User)
async def get_user(user_id: UUID4) -> User | JSONResponse:
    try:
        return next((user for user in User.__users__ if user.id == user_id))
    except StopIteration:
        return JSONResponse(status_code=404, content={"message": "User not found"})

Na função main temos um cliente de teste para executar os testes

In [20]:
def main() -> None:
    with TestClient(app) as client:
        for i in range(5):
            response = client.post(
                "/users",
                json={"name": f"User {i}", "email": f"example{i}@arjancodes.com"},
            )
            assert response.status_code == 200
            assert response.json()["name"] == f"User {i}", (
                "The name of the user should be User {i}"
            )
            assert response.json()["id"], "The user should have an id"

            user = User.model_validate(response.json())
            assert str(user.id) == response.json()["id"], "The id should be the same"
            assert user.signup_ts, "The signup timestamp should be set"
            assert user.friends == [], "The friends list should be empty"
            assert user.blocked == [], "The blocked list should be empty"

        response = client.get("/users")
        assert response.status_code == 200, "Response code should be 200"
        assert len(response.json()) == 5, "There should be 5 users"

        response = client.post(
            "/users", json={"name": "User 5", "email": "example5@arjancodes.com"}
        )
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "The name of the user should be User 5"
        )
        assert response.json()["id"], "The user should have an id"

        user = User.model_validate(response.json())
        assert str(user.id) == response.json()["id"], "The id should be the same"
        assert user.signup_ts, "The signup timestamp should be set"
        assert user.friends == [], "The friends list should be empty"
        assert user.blocked == [], "The blocked list should be empty"

        response = client.get(f"/users/{response.json()['id']}")
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "This should be the newly created user"
        )

        response = client.get(f"/users/{uuid4()}")
        assert response.status_code == 404
        assert response.json()["message"] == "User not found", (
            "We technically should not find this user"
        )

        response = client.post("/users", json={"name": "User 6", "email": "wrong"})
        assert response.status_code == 422, "The email address is should be invalid"

if __name__ == "__main__":
    main()